In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [2]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 50
BS = 128

In [3]:
DIRECTORY = "OCR_Handwritten_dataset"
CATEGORIES = os.listdir(DIRECTORY)

In [4]:
print(CATEGORIES)

['0', 'Q', 'n_', 'B', 'b_', '2', 'i_', '1', 'v_', 'A', '7', 'S', 'f_', 'K', 'u_', '4', 't_', 'C', 'F', 'L', 'p_', 'H', 'o_', 'r_', 'h_', 'z_', 'k_', 'E', 'I', 'm_', 'j_', 'T', 'c_', 'P', 'R', '3', '8', 'D', 'O', 'g_', 'e_', 'G', 'a_', 'X', 'w_', 'U', 'q_', 'Y', 'V', 'W', '9', 'l_', 'd_', 'x_', 's_', '6', 'M', 'Z', '5', 'y_', 'N', 'J']


In [5]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []
str='_'

[INFO] loading images...


In [6]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(98, 98))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)

In [7]:
import pandas as pd
df = pd.DataFrame(labels)

In [8]:
labels = pd.get_dummies(df)

In [11]:
labels = np.array(labels)
data = np.array(data, dtype="float32")

In [12]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [13]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(98, 98, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(62, activation='softmax'))

In [15]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [16]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] training head...
Train for 226 steps, validate on 7241 samples
Epoch 1/50
226/226 [==============================] - 84s 373ms/step - loss: 3.8973 - accuracy: 0.0438 - val_loss: 3.6395 - val_accuracy: 0.0767
Epoch 2/50
226/226 [==============================] - 81s 359ms/step - loss: 3.6778 - accuracy: 0.0744 - val_loss: 3.4190 - val_accuracy: 0.1126
Epoch 3/50
226/226 [==============================] - 79s 351ms/step - loss: 3.5267 - accuracy: 0.0968 - val_loss: 3.2767 - val_accuracy: 0.1470
Epoch 4/50
226/226 [==============================] - 80s 356ms/step - loss: 3.4235 - accuracy: 0.1139 - val_loss: 3.1522 - val_accuracy: 0.1627
Epoch 5/50
226/226 [==============================] - 80s 353ms/step - loss: 3.3428 - accuracy: 0.1213 - val_loss: 3.0731 - val_accuracy: 0.1738
Epoch 6/50
226/226 [==============================] - 80s 354ms/step - loss: 3.2797 - accuracy: 0.1344 - val_loss: 3.0225 - val_accuracy: 0.1676
Epoch 7/50
226/226 [==============================] - 80s 35

In [17]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("OCR_handwritten ver 2.0.model", save_format="h5")

[INFO] saving mask detector model...
